In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import string
import random
import pandas as pd

In [66]:
random.seed(123)

#Print user inputs
def startingLine(numNodes, networkConnectivity, stPerimeter, stDefender, stAttacker):
    print("The attack occured under the following parameters:")
    print("Number of nodes: " + str(numNodes))
    print("Network connectivity: " + str(networkConnectivity))
    print("Perimeter strength: " + str(stPerimeter))
    print("Defender strength: " + str(stDefender))
    print("Attacker strength: " + str(stAttacker) + "\n")
        

#function takes in input variables lst (list of nodes) and pairs (list of pairs)
#returns a dictionary of nodes and neighbors
def getNeighbors(lst, pairs):
    d={}
    keys=lst
  
    for node in lst:
        neighbors=[]
        for pair in pairs:
            if node in pair:
                neighbors+=[n for n in pair if n!=node]
        key=node
        values=list(set(neighbors))
        d[key] = values
    return d
            

#Function takes in input lst (list of nodes) 
#returns a table with Node, Neighbors, and State (all with state 1 as healthy)
def createTable(lst):
    d=getNeighbors(lst, pairs)
    nodes=list(d.keys())
    neighbors=list(d.values())
    table=pd.DataFrame()
    table['Node']=nodes
    table['Neighbors']=0
    table['Neighbors']=table['Neighbors'].astype('object')
    table['Neighbors']=neighbors
    table['State']=1
    table=table.set_index('Node')
    return table

#Updates table
def updateTable(numNodes, stPerimeter, stDefender, stAttacker, Time, Table):
    #iterate through each node
    for index, row in table.iterrows():
        prob_survival = np.e**(-lmbda*time)
        neighbors=row[0]
        state=row[1]
        
        #don't adjust lambda if node has no neighbors
        if not neighbors:
            prob_survival=prob_survival*1 
        
        
        else:
            infected_neighbors=0
            for neighbor in neighbors:
                if table.iloc[neighbor-1][1]==1:
                    infected_neighbors+=1                        
            #halve the survival probability for each infected neighbor
            prob_survival=prob_survival*((1/2)**infected_neighbors) 
                
        #run a bernoulli trial with prob_survival to get new status
        new_status=np.random.binomial(size=1, n=1, p=prob_survival)[0]
        Table.loc[index,'State']=new_status 
        
    #show results      
    percent_infected=(Table['State'] == 0).sum()*100 / numNodes
    percent_healthy=(Table['State'] == 1).sum() * 100 /numNodes
    print("----------------------------------------------------")
    print("At time t=" + str(time))
    print(str(percent_infected) + "% of nodes are now infected.")
    print(str(percent_healthy) + "% of nodes are still healthy.")

    #color nodes
    color_map = []
    for node in network:
        if Table.loc[node,'State']==0:
            color_map.append('red')
        else: 
            color_map.append('green') 

    nx.draw(network, node_color=color_map, with_labels = True)
    plt.show()
    return Table

In [67]:
random.seed(123)

numNodes=10
nodes_lst=[i+1 for i in range(numNodes)]
network = nx.Graph()
for x in range(1, numNodes + 1):
    network.add_node(x)

central_nodes = [1,2,3]
regular_nodes= [4,5,6,7,8,9,10]
for regular_node in regular_nodes:
    for central_node in central_nodes:
        pair = central_node, regular_node
        pairs.append(pair)
network.add_edges_from(pairs)

table=createTable(nodes_lst)
time=0

check = "Y"
while  check == "Y" or check == "YES":
    check = input("Continue with simulation? (Y/N): ")
    check = check.upper()
    if (check == "Y" or check == "YES") != True :
        break

    
    '''
        Run Network Simulation
    '''

    stPerimeter = int(input("Input perimeter strength (1-10, 10 being strongest): "))
    stDefender = int(input("Input defender strength (1-10, 10 being strongest): "))
    stAttacker = int(input("Input attacker strength (1-10, 10 being strongest): "))
    showStartNet = input("View initial network? (Y/N) ")
    showNet = input("View static network? (Y/N) ")

    if showStartNet == "Y" or showStartNet == "y":
        #Draw starting network
        print("At t=0:")
        print("100% of nodes are healthy.")
        nx.draw(network, node_color='green', seed=123, with_labels = True)
        plt.show()

    if showNet == "Y" or showNet == "y":
        exp_rate=2
        rate=exp_rate * stPerimeter * (stDefender/stAttacker)
        lmbda=1/rate
        
        time+=1
        table=updateTable(10, stPerimeter, stDefender, stAttacker, time, table)
        


KeyboardInterrupt: Interrupted by user